In [1]:
import requests
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_html_from_pmc(pmc_id: str) -> BeautifulSoup:
    """Descarga el HTML del artículo de PMC y lo devuelve como BeautifulSoup."""
    url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc_id}/"
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                      "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    try:
        response = requests.get(url, headers=headers, timeout=15)
        time.sleep(random.uniform(1, 3))  # Pausa para evitar bloqueo
        if response.status_code == 200:
            return BeautifulSoup(response.text, "html.parser")
        else:
            print(f"⚠️ Error {response.status_code} al descargar {pmc_id}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al conectar con {pmc_id}: {e}")
        return None


def extract_images(soup: BeautifulSoup, base_url: str) -> list:
    """Extrae enlaces de imágenes relevantes del artículo."""
    if not soup:
        return []

    images = []
    for img in soup.find_all("img"):
        src = img.get("src")
        if src and not src.startswith("data:"):  # evita iconos base64
            full_url = urljoin(base_url, src)
            # Filtrar imágenes irrelevantes (logotipos, íconos)
            if "figure" in full_url or "bin" in full_url:
                images.append(full_url)

    return images


def process_article(pmc_id: str) -> dict:
    """Devuelve solo el PMC_ID y la lista de imágenes."""
    base_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc_id}/"
    soup = get_html_from_pmc(pmc_id)
    if not soup:
        return {"PMC_ID": pmc_id, "Images": []}

    return {
        "PMC_ID": pmc_id,
        "Images": extract_images(soup, base_url)
    }


def process_csv(csv_file: str, output_json: str = "articles_images.json") -> list:
    """Procesa los PMC_ID del CSV y genera un JSON con solo imágenes."""
    df = pd.read_csv(csv_file)
    all_data = []

    for _, row in df.iterrows():
        pmc_id = row["PMC_ID"]
        print(f"🔎 Procesando imágenes de {pmc_id} ...")
        article_data = process_article(pmc_id)
        all_data.append(article_data)

    pd.DataFrame(all_data).to_json(output_json, orient="records", indent=4)
    print(f"✅ Resultados guardados en {output_json}")
    return all_data

In [3]:
import pandas as pd
import json

# ======================================
# Iterar sobre los IDs del CSV y procesar
# ======================================
def process_all_articles_from_csv(csv_file: str, output_json: str = "articles_images.json", output_csv: str = "articles_images.csv"):
    """
    Lee un archivo CSV que contiene una columna 'PMC_ID',
    procesa cada artículo llamando a process_article(pmc_id)
    y guarda los resultados en JSON y CSV.
    """
    # Leer CSV de entrada
    df = pd.read_csv(csv_file)

    # Lista para almacenar todos los resultados
    all_data = []

    # Iterar sobre cada PMC_ID del CSV
    for pmc_id in df["PMC_ID"]:
        print(f"Procesando {pmc_id} ...")
        article_data = process_article(pmc_id)  # Aquí se llama tu función
        all_data.append(article_data)

    # Guardar resultados en un archivo JSON
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(all_data, f, indent=4, ensure_ascii=False)

    # Guardar resultados en un archivo CSV
    pd.DataFrame(all_data).to_csv(output_csv, index=False, encoding="utf-8")

    return all_data


# ==========================
# Punto de entrada principal
# ==========================
if __name__ == "__main__":
    # Cambia este nombre por tu archivo CSV real
    csv_input = "articles_with_pmcid.csv"

    articles = process_all_articles_from_csv(csv_input)

    print(f"Se procesaron {len(articles)} artículos correctamente ✅")

Procesando PMC4136787 ...
Procesando PMC3630201 ...
Procesando PMC11988870 ...
Procesando PMC7998608 ...
Procesando PMC5587110 ...
Procesando PMC8396460 ...
Procesando PMC5666799 ...
Procesando PMC5460236 ...
Procesando PMC6222041 ...
Procesando PMC6813909 ...
Procesando PMC4095884 ...
Procesando PMC3040128 ...
Procesando PMC3177255 ...
Procesando PMC11500582 ...
Procesando PMC5387210 ...
Procesando PMC4642138 ...
Procesando PMC5387210 ...
Procesando PMC2915878 ...
Procesando PMC3901686 ...
Procesando PMC6985101 ...
Procesando PMC6387434 ...
Procesando PMC6371294 ...
Procesando PMC7072278 ...
Procesando PMC8441986 ...
Procesando PMC9400218 ...
Procesando PMC9267413 ...
Procesando PMC9576569 ...
Procesando PMC10789781 ...
Procesando PMC10772081 ...
Procesando PMC11166946 ...
Procesando PMC11166944 ...
Procesando PMC11166968 ...
Procesando PMC7000411 ...
Procesando PMC7787258 ...
Procesando PMC8716943 ...
Procesando PMC4826010 ...
Procesando PMC6048781 ...
Procesando PMC7010715 ...
Proce